In [1]:
from jupyterthemes import jtplot
jtplot.style()

import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import sys

# Self Defined Functions
sys.path.append('..\src')

from findiff import *
from opt import *
from KRot import *

# Gradient Flow

In [2]:
#GRID
n = 20*6
h=1/(n-1)
dt=1
x = np.linspace(0,1,n)

#DISTRIBUTIONS
rho= np.zeros(n)
rho[n//10] = 1
sigma = np.zeros(n)
sigma[9*n // 10] = 1

f = sigma-rho

u = np.zeros(n)

#DIFFERENCE OPERATORS
D = D1FE(n, bc = 'Neumann', matrix=True)

u = dt*(f-D@u)+u
plt.figure()
plt.plot(x,u) 

# ADMM

In [3]:
#GRID
n = 20*6
h=1/(n-1)
dt=1
x = np.linspace(0,1,n)

#DISTRIBUTIONS
rho= np.zeros(n)
rho[n//10] = 1
sigma = np.zeros(n)
sigma[9*n // 10] = 1

#VARIABLES
lam1=np.ones(n)
lam2=np.ones(n)
mu1=1e-3
mu2=1e-3
c=np.ones(n)

#DIFFERENCE OPERATORS
D = D1FE(n, bc = 'Neumann', matrix=True)/h
D2 = DD1CD(n,bc='Neumann',matrix=True)/h**2

#INITIAL GUESS
phi = np.sin(np.pi*x)
d = D@phi
u = np.zeros(n)

plt.figure(figsize=(16,10))
plt.subplot(231)
plt.title('Distributions')
plt.bar(x,rho-sigma,2*h)
plt.subplot(232)
plt.plot(x,lam1)
plt
plt.subplot(233)
plt.plot(x,lam2)
plt.subplot(234)
plt.plot(x,phi)
plt.subplot(235)
plt.plot(x,d)
plt.subplot(236)
plt.plot(x,u)

In [4]:
#UPDATE PHI
phi = cp.Variable(n)
prob = cp.Problem(cp.Minimize(cp.sum_squares(mu2/2*D2@phi-(sigma-rho)-D@lam2+mu2*D@d)))
prob.solve()
phi = phi.value
plt.plot(phi)

In [5]:
#UPDATE D
Gphi = D@phi
d = cp.Variable(n)
prob = cp.Problem(cp.Minimize((lam1+mu1*(u-c))@cp.abs(d)+(mu1+mu2)/2*cp.norm(d,2)-mu2*d@Gphi))
prob.solve()
d = d.value
plt.plot(x,d)

In [6]:
#UPDATE U
u = np.abs(d)-c
u=u.clip(min=0)
plt.plot(x,u)

In [7]:
lam1 = lam1+mu1*(np.abs(d)+u-c)
lam2 = lam2+mu2*(d-Gphi)
plt.plot(x,lam1)
plt.plot(x,lam2)

In [8]:
itrs = 2

#GRID
n = 20*6
h=1/(n-1)
dt=1
x = np.linspace(0,1,n)

#DISTRIBUTIONS
rho= np.zeros(n)
rho[n//10] = 1
sigma = np.zeros(n)
sigma[9*n // 10] = 1

#VARIABLES
lam1=np.ones(n)
lam2=np.ones(n)
mu1=1e-5
mu2=1
c=np.ones(n)

#DIFFERENCE OPERATORS
D = D1FE(n, bc = 'Neumann', matrix=True)/h
D2 = DD1CD(n,bc='Neumann',matrix=True)/h**2

#INITIAL GUESS
phi = np.sin(np.pi*x)
d = D@phi
u = np.zeros(n)

#PLOT INITIAL
plt.figure(figsize=(16,10))
plt.subplot(231)
plt.title('Distributions')
plt.bar(x,rho-sigma,2*h)
plt.subplot(232)
plt.plot(x,lam1)
plt.title('$\lambda_1$')
plt.subplot(233)
plt.plot(x,lam2)
plt.title('$\lambda_2$')
plt.subplot(234)
plt.plot(x,phi)
plt.title('$\phi$')
plt.subplot(235)
plt.plot(x,d)
plt.title('$d$')
plt.subplot(236)
plt.plot(x,u)
plt.title('$u$')

for i in range(itrs):

    #UPDATE PHI
    # DL/DPHI = STEP UPDATE
    phi = cp.Variable(n)
    prob = cp.Problem(cp.Minimize(cp.sum_squares(mu2/2*D2@phi-(sigma-rho)-D@lam2+mu2*D@d)))
    prob.solve()
    phi = phi.value

    #UPDATE D
    Gphi = D@phi
    d = cp.Variable(n)
    #lam1@(|d|+u-c)+mu1/2|| |d|+u-c||^2 + lam2@(d-Gphi)+mu2/2|| d-Gphi||^2
    prob = cp.Problem(cp.Minimize((lam1+mu1*(u-c))@cp.abs(d)+lam2@d+(mu1+mu2)/2*cp.sum_squares(d)-mu2*d@Gphi))
    prob.solve()
    d = d.value

    #UPDATE U
    #|d|+u-c=0
    u = np.abs(d)-c
    u=u.clip(min=0)

    #UPDATE LAMBDA
    lam1 = lam1+mu1*(np.abs(d)+u-c)
    lam2 = lam2+mu2*(d-Gphi)

    plt.figure(figsize=(16,10))
    plt.subplot(231)
    plt.plot(x,phi)
    plt.title('$\phi$')
    plt.subplot(232)
    plt.plot(x,Gphi)
    plt.title('$\\nabla\phi$')
    plt.subplot(233)
    plt.plot(x,d)
    plt.title('d')
    plt.subplot(234)
    plt.plot(x,u)
    plt.title('u')
    plt.subplot(235)
    plt.plot(x,lam1)
    plt.title('$\lambda_1$')
    plt.subplot(236)
    plt.plot(x,lam2)
    plt.title('$\lambda_2$')